In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pathlib import Path
import json
import random

# Import API key
from api_keys import owm

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


Bad key text.latex.preview in file C:\Users\Dan\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\Dan\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\Users\Dan\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpeg is saved, the de

In [2]:
# Generate a random integer within the ranges
# lat = x long = y
def rand_lat():
    lat = random.randint(lat_range[0], lat_range[1])
    return lat

def rand_lng():
    lng = random.randint(lng_range[0], lng_range[1])
    return lng

In [3]:
# Define a function to generate a city from the random coordinates, in the format of a list with country code, lat and lng
def rand_city():
    x = rand_lat()
    y = rand_lng()
    city_code = citipy.nearest_city(x, y)
    city = city_code.city_name
    country = city_code.country_code
    output = [city, country, x, y]
    return output

In [4]:
rand_city()

['ushuaia', 'ar', -57, -67]

In [91]:
# Creating an empty dictionary to build the dataframe with
dict = {
    'City': [],
    'Country': [],
    'Latitude': [],
    'Longitude': []
}
# For loop to fill the dictionary with 500 cities
rows = 500
for i in range(rows):
    city_list = rand_city()
    if city_list[0] in dict['City']:
        
        continue
    dict['City'].append(city_list[0])
    dict['Country'].append(city_list[1])
    dict['Latitude'].append(city_list[2])
    dict['Longitude'].append(city_list[3])
# Turning the dictionary into a dataframe
city_df = pd.DataFrame(dict)
city_df.head()

,City,Country,Latitude,Longitude
0,qui nhon,vn,14,109
1,kavaratti,in,11,64
2,isafjordur,is,65,-26
3,happy valley-goose bay,ca,55,-57
4,mejit,mh,12,178


In [95]:
# Creating an empty dictionary to build the dataframe with
dict = {
    'City': [],
    'Country': [],
    'Latitude': [],
    'Longitude': []
}
# While loop to fill the dictionary with 500 cities
rows = 0
while rows < 500:
    city_list = rand_city()
    if city_list[0] in dict['City']:
        continue
    dict['City'].append(city_list[0])
    dict['Country'].append(city_list[1])
    dict['Latitude'].append(city_list[2])
    dict['Longitude'].append(city_list[3])
    rows += 1
# Turning the dictionary into a dataframe
city_df = pd.DataFrame(dict)
city_df.head()

,City,Country,Latitude,Longitude
0,fuerte olimpo,py,-21,-58
1,lamu,ke,-5,45
2,san pedro pochutla,mx,10,-97
3,longyearbyen,sj,90,3
4,port-aux-francais,tf,-84,76


In [96]:
# Store the dataframe as a local csv
city_df.to_csv('output_data/WeatherAPI_Cities.csv', index=False, mode='w')

In [10]:
w_df = pd.read_csv('output_data/WeatherAPI_Cities.csv')
w_df.head()

,City,Country,Latitude,Longitude
0,fuerte olimpo,py,-21,-58
1,lamu,ke,-5,45
2,san pedro pochutla,mx,10,-97
3,longyearbyen,sj,90,3
4,port-aux-francais,tf,-84,76


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).

> **HINT:** The OpenWeatherMap API only allows 60 calls per minute under their free plan. Try using `time.sleep(60)` after each set of 50 cities to avoid API errors due to large calls.

In [22]:
# Save config information
url = "https://api.openweathermap.org/data/2.5/onecall?"
lat = str(w_df['Latitude'])
lon = str(w_df['Longitude'])
# Build query URL
query_url = url + "lat=" + lat + "&lon=" + lon + "&exclude=hourly,daily,minutely,alerts&" + "appid=" + owm

In [21]:
# Get weather data


The weather API responded with: {'lat': -21, 'lon': -58, 'timezone': 'America/Asuncion', 'timezone_offset': -14400, 'current': {'dt': 1692472966, 'sunrise': 1692439935, 'sunset': 1692481168, 'temp': 303.46, 'feels_like': 303.2, 'pressure': 1007, 'humidity': 40, 'dew_point': 288.36, 'uvi': 2.71, 'clouds': 0, 'visibility': 10000, 'wind_speed': 0.63, 'wind_deg': 130, 'wind_gust': 0.52, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}]}}.


In [36]:
# Lists to store API results
temperatures = []
humidities = []
cloudinesses = []
windspeeds = []

# Loop through each row and make API calls
for index, row in w_df.iterrows():
    city = row['City']
    lat = str(row['Latitude'])
    lon = str(row['Longitude'])
    
    query_url = f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=hourly,daily,minutely,alerts&appid={owm}"
    
    # Make API call
    weather_response = requests.get(query_url)
    weather_json = weather_response.json()
    
    temp = weather_json['current']['temp']
    hum = weather_json['current']['humidity']
    cloud = weather_json['current']['clouds']
    wind = weather_json['current']['wind_speed']
    
    # Delay after every 50 calls
    if (index + 1) % 50 == 0:
        time.sleep(60)
    
    # Append results to lists
    temperatures.append(temp)
    humidities.append(hum)
    cloudinesses.append(cloud)
    windspeeds.append(wind)
    
    print(f'City # {(index + 1)} : {city}')

# Add results to dataframe
w_df['Temperature'] = temperatures
w_df['Humidity'] = humidities
w_df['Cloudiness'] = cloudinesses
w_df['Windspeed'] = windspeeds

City # 1 : fuerte olimpo
City # 2 : lamu
City # 3 : san pedro pochutla
City # 4 : longyearbyen
City # 5 : port-aux-francais
City # 6 : chibougamau
City # 7 : port douglas
City # 8 : ilulissat
City # 9 : juan griego
City # 10 : adamstown
City # 11 : vorgashor
City # 12 : qaqortoq
City # 13 : malyye derbety
City # 14 : whitehorse
City # 15 : ushuaia
City # 16 : blackmans bay
City # 17 : sinabang
City # 18 : wajir
City # 19 : saint john
City # 20 : flying fish cove
City # 21 : sevastopol
City # 22 : wailua homesteads
City # 23 : thunder bay
City # 24 : stanley
City # 25 : avarua
City # 26 : boguchany
City # 27 : west island
City # 28 : saipan
City # 29 : ola
City # 30 : grytviken
City # 31 : invercargill
City # 32 : halmstad
City # 33 : vingt cinq
City # 34 : al bawiti
City # 35 : leca da palmeira
City # 36 : waitangi
City # 37 : keflavik
City # 38 : biaora
City # 39 : mhamid
City # 40 : daru
City # 41 : tromso
City # 42 : port augusta
City # 43 : zambezi
City # 44 : port mathurin
City # 

In [37]:
w_df.tail()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed
495,bel ombre,sc,-4,54,299.02,88,54,6.97
496,gobabis,na,-21,19,296.87,13,0,3.18
497,jaicos,br,-7,-41,307.82,32,63,4.92
498,son la,vn,21,104,292.66,99,100,0.22
499,gonzaga,ph,18,122,296.12,98,68,0.21


In [ ]:
from pprint import pprint

url = 
json = response.json()
pprint(json)